In [6]:
import pandas as pd
import numpy as np
import os
import joblib as joblib
import glob

from tqdm.auto import tqdm
from datetime import datetime
from scipy.stats import zscore

# from ReMASTER.system import get_data_dir

tqdm.pandas()

In [12]:
file_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ_5Years_8_11_2024.csv'

# Initialize an empty DataFrame for the combined data
# data = pd.DataFrame()
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect its structure
data.head()

,Time,Open,High,Low,Close,Volume
0,8/11/2019 23:05,9073.25,9098.50,9073.00,9092.50,1758
1,8/11/2019 23:10,9093.25,9095.50,9089.75,9092.75,438
2,8/11/2019 23:15,9093.00,9096.25,9088.00,9089.75,590
3,8/11/2019 23:20,9090.25,9090.25,9086.00,9087.00,278
4,8/11/2019 23:25,9086.75,9088.25,9079.75,9083.75,711


In [16]:
# Step 1: Convert 'Time' column to datetime format
data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y %H:%M')

# Step 2: Check for any NA values and drop columns with NA values (if any)
data.dropna(axis=1, inplace=True)

# Step 3: Perform robust daily Z-score normalization on each feature dimension
# Extract date component for daily grouping
data['Date'] = data['Time'].dt.date
# Group by 'Date' and normalize 'Open', 'High', 'Low', 'Close', and 'Volume' columns
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data[feature_columns] = data.groupby('Date')[feature_columns].transform(zscore)

# Step 4: Drop 5% of the most extreme values from the 'Close' column to reduce label outliers
# Identify upper and lower 2.5% quantiles for 'Close'
q_low, q_high = data['Close'].quantile([0.025, 0.975])
data = data[(data['Close'] >= q_low) & (data['Close'] <= q_high)]

# Step 5: Drop NA rows (if any remain after filtering) and reset index
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

# Step 6: Save the cleaned data to a new CSV file
# Create the directory if it doesn't exist
save_path = 'new_data/reshaped_data.csv'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
data.to_csv(save_path, index=False)

# Display first few rows of the cleaned data to confirm
data.head()

/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_8889/1563669071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y %H:%M')
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_8889/1563669071.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(axis=1, inplace=True)
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_8889/1563669071.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,Time,Open,High,Low,Close,Volume,Date
0,2019-08-11 23:05:00,-2.256861,1.802600,-2.130639,1.565561,2.904046,2019-08-11
1,2019-08-11 23:10:00,1.334527,1.162968,1.555467,1.634445,-0.020545,2019-08-11
2,2019-08-11 23:15:00,1.289635,1.322876,1.170351,0.807829,0.316226,2019-08-11
3,2019-08-11 23:20:00,0.795819,0.043611,0.730219,0.050098,-0.375040,2019-08-11
4,2019-08-11 23:25:00,0.167326,-0.382810,-0.645194,-0.845403,0.584314,2019-08-11
